In [131]:
import os 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import mne 
import torch
import torch.nn as nn
import math
from scipy.interpolate import interp1d
from scipy.stats import iqr 

from typing import Optional, Dict, Any, Tuple

from lightning.pytorch.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

from torch.utils.data import ConcatDataset, Dataset, DataLoader, RandomSampler


In [3]:
def is_integer_with_custom_precision(num: float, eps: float = 1e-6, int_value: Optional[int] = None):
    if int_value is None:
        int_value = int(num)
    return abs(int_value - num) < eps, int_value



In [4]:
EEG_SAMPLING_RATE = 250
FMRI_TR = 2.0  # sec

RUNS = [
    ('rs', '01'),
    ('rs', '02'),
    ('rs', '03'),
    ('rs', '04'),
    ('rs', '05'),
    ('rs', '06'),
    ('socword', '01'), 
    ('socword', '02'),
    ('socword', '03'),
    ('socword', '04'),
    ('socword', '05'),
    ('socword', '06')
    ]
RUNSNATVIEW = [
    ('01', 'checker'),
    ('01', 'dme_run-01'),
    ('01', 'dme_run-02'),
    ('01', 'inscapes'),
    ('01', 'monkey1_run-01'),
    ('01', 'monkey1_run-02'),
    ('01', 'rest'),
    ('01', 'tp_run-01'),
    ('01', 'tp_run-02')
]

RUNS_1895 = [
    ('rs', '01'),
    ('rs', '02'),
    ('socword', '01'), 
    ('socword', '02'),
    ('socword', '03')
    ]
# IN SAMPLES WITH 2 Hz SAMPLING RATE. FOR SECONDS DIVIDE BY 2
TEST_SIZES = {
    'checker': 100,
    'dme': 250,
    'inscapes': 250,
    'monkey1': 150,
    'rest': 250,
    'tp': 100,
    'dmh': 250,
    'monkey2': 150,
    'monkey5': 150
}


In [5]:

def load_natdataset_eeg_data(root: str, sub: str, ses: str, task: str, eeg_channels: Optional[list[str]]):
    """
    Loads EEG data, checks it and crops by first fMRI scan

    Parameters
    ----------
    root : str
        A directory where the dataset is
    sub : str
        A subject id
    ses : str
        A session id
    task : str
        A task id
    eeg_channels : list[str] or None
        A list of EEG channels to load from file. If None, extracts every channel available

    Returns
    -------
    df_eeg : pd.DataFrame
        A DataFrame with EEG data. Columns are EEG channels
    eeg_channels : list[str]
        A list of EEG channels loaded from file
    vhdr_path : str
        A path to the .vhdr file from which the data was extracted
    eeg_times : np.ndarray
        Time-steps of the EEG signal
    """
    filename = f'sub-{sub}/ses-01/eeg/sub-{sub}_ses-{ses}_task-{task}_eegMRbvCBbviiR250.vhdr'
    vhdr_path = os.path.join(root, filename)
    eeg_raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
    eeg_raw = eeg_raw.pick('eeg')

    events, event_ids = mne.events_from_annotations(eeg_raw)

    r128_scans = events[events[:, -1] == event_ids['Stimulus/R128']]

    unique_scan_diffs, counts_diffs = np.unique(np.diff(r128_scans[:, 0]), return_counts=True)
    assert len(unique_scan_diffs) == 1, (unique_scan_diffs, counts_diffs)
    assert unique_scan_diffs[0] == 525  # 2.1 sec (fMRI TR) * 250 Hz (EEG sampling rate)

    assert eeg_raw.info['sfreq'] == EEG_SAMPLING_RATE
    first_r128 = r128_scans[0, 0] / EEG_SAMPLING_RATE

    cropped_eeg = eeg_raw.crop(tmin=first_r128)

    df_eeg = cropped_eeg.to_data_frame()

    if eeg_channels is None:
        eeg_channels = [column for column in df_eeg.columns if column != 'time']

    eeg_times = df_eeg['time']
    df_eeg = df_eeg[eeg_channels]

    return df_eeg, eeg_channels, vhdr_path, eeg_times, events, event_ids


In [6]:
def load_eeg_data(root: str, ses:str, run:str, task: str, eeg_channels: Optional[list[str]]):

    filename = f'sub-101472_ses-{ses}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'
    eeg_vhdr_path = os.path.join(root, f'ses-{ses}', 'eeg', filename)

    
    eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
    events, event_ids = mne.events_from_annotations(eeg_raw)
    events_dict_cut = {'Scanner/Scan Start': event_ids['Scanner/Scan Start']}  # use only Scan Start event


    try:
        scan_key = 'Scanner/Scan Start'
        segment_scans = events[events[:, -1] == event_ids[scan_key]]
    except: 
        scan_key = 'Stimulus/Scan Start'
        segment_scans = events[events[:, -1] == event_ids[scan_key]] 
        print(f'ERROR: FAULTY EVENT NAME. PATH:{eeg_vhdr_path}')


    unique_scan_diffs, counts_diffs = np.unique(np.diff(segment_scans[:, 0]), return_counts=True)
    assert len(unique_scan_diffs) == 1, (unique_scan_diffs, counts_diffs)
    assert unique_scan_diffs[0] == 3000  

    assert eeg_raw.info['sfreq'] == EEG_SAMPLING_RATE
    first_segment = segment_scans[0, 0] / EEG_SAMPLING_RATE

    cropped_eeg = eeg_raw.crop(tmin=first_segment)
    df_eeg = cropped_eeg.to_data_frame()

    if eeg_channels is None:
        eeg_channels = [column for column in df_eeg.columns if column != 'time']

    eeg_times = df_eeg['time']


    epochs = mne.Epochs(
        cropped_eeg,
        events = events, 
        event_id =events_dict_cut, 
        tmin = 6.5,
        tmax = 11.5-1/EEG_SAMPLING_RATE, 
        baseline = None, 
        preload = True
    )

    epochs_df = epochs.to_data_frame()
    #try: 
    assert (epochs_df['epoch'].unique() == np.where(events[:,2] == 10002)).all()
    #except: 
        #assert (epochs_df['epoch'].unique() == np.where(events[:,2] == 10001)).all()
        #print(f'FAULTY EVENT NAMES. SESSION:{ses},RUN:{run},  TASK:{task}')
    epochs_array = epochs.get_data()[:-1, ...]
    segment_scans = segment_scans[:-1]


    return epochs_array, eeg_channels, eeg_vhdr_path, eeg_times, events,  event_ids, segment_scans

In [7]:
#UPDATE FUNCTIONS BELOW FOR THE EPOCHS RETURN + ALIGN EPOCHT WITH FMRI  

In [8]:
def load_fmri_data(root, ses, task, run, roi_folder, rois ): 
    """
    Loads pre-processed fMRI data. The original TR is 2.0 
    """
    filename = f'sub-101472_ses-{ses}_task-{task}_run-{run}.csv'
    csv_path = os.path.join(root, f'ses-{ses}', roi_folder, filename)
    fmri_data = pd.read_csv(csv_path)

    if rois is None:
        rois = [column for column in fmri_data.columns if column not in ['Nvol', ' Tstart']]
    # assert for tsart integer 
    assert all([i.is_integer() for i in fmri_data[' Tstart']])
    fmri_times = fmri_data[' Tstart'].to_numpy()

    fmri_times = fmri_times.astype(int)
    fmri_times = fmri_times//12

    fmri_data = fmri_data.assign(N_cycle=pd.Series(fmri_times).values)
    df_ncycle = fmri_data.groupby(by = 'N_cycle')

    fmri_array = np.full((len(df_ncycle), len(rois), 3), fill_value = np.inf)

    for idx, group in df_ncycle: 
        group = group[rois].to_numpy()
        fmri_array[idx, ...] = group.T 

    # check if no inf that were previously filled are left  
    assert not np.isinf(fmri_array).any(), (f'') 

    # cut the first triplet 
    fmri_array = fmri_array[1:, ...]

    # if averaging for 3 scans; fmri_array = np.mean(fmri_array, axis = -1)
    # better move to pytorch dataset 

    return fmri_array, fmri_times, rois, csv_path, fmri_data



In [9]:
def load_data(root: str, run: str, task: str, ses:str,
              rois: Optional[list[str]], eeg_channels: Optional[list[str]],
              sampling_rates_ratio = 500, roi_folder: str = 'roi'):
    
    fmri_array, fmri_times, rois, fmri_path, fmri_data = load_fmri_data(
        root=root, run=run, ses=ses, task=task, rois=rois, roi_folder=roi_folder
    )

    epochs_array, eeg_channels, eeg_path, eeg_times, events, event_ids, segment_scans = load_eeg_data(
        root=root, ses=ses, run=run, task=task, eeg_channels=eeg_channels
    )

    assert (fmri_array.shape[0]) == len(segment_scans), (f'EEG segment marks should be 3x '
                                                 f'fMRI sample size. '
                                                 f'fMRI sample size is {(fmri_array.shape[0])}. '
                                                 f'Number of segment marks: {len(segment_scans)} '
                                                 f'file is Root: {root},  ses: {ses}, run: {run},'
                                                 f' task: {task}.')
    


   

    # assert epochs.shape[0] == (df_fmri.shape[0] * sampling_rates_ratio), (f'EEG size: {df_eeg.shape}, fMRI '
    #                                                                       f'size: {df_fmri.shape}, sr ratio: '
    #                                                                       f'{sampling_rates_ratio}.\nRoot: {root},  '
    #                                                                       f'sub: {sub}, ses: {ses}, task: {task}.')







    #fmri_for_global_trend = df_fmri[rois_for_global_trend].to_numpy().T



    # center BOLD around 0
    fmri_array = fmri_array - np.mean(fmri_array, axis=1, keepdims=True)
    #fmri_for_global_trend = fmri_for_global_trend - np.mean(fmri_for_global_trend, axis=1, keepdims=True)

    # if separate_global_trend:
    #     assert fmri_array.ndim == 2, fmri_array.shape
    #     assert fmri_for_global_trend.ndim == 2, fmri_for_global_trend.shape
    #     global_trend_array = np.mean(fmri_for_global_trend, axis=0)

    #     inner = np.inner(global_trend_array, global_trend_array)
    #     outer = np.outer(global_trend_array, global_trend_array)

    #     detrend = fmri_array @ outer / inner
    #     fmri_array = fmri_array - detrend

    #     for idx in range(fmri_array.shape[0]):
    #         corr_coeff = np.corrcoef(fmri_array[idx, :], global_trend_array)[0, 1]
    #         assert abs(corr_coeff) < 1e-6, (
    #             f'Sub: {sub}, ses: {ses}, task: {task}, ROI: {rois[idx]} has correlation above threshold: {corr_coeff}'
    #         )

    #     fmri_array = np.concatenate((fmri_array, global_trend_array[None, :]), axis=0)
    #     assert np.equal(fmri_array[-1], global_trend_array).all()

    # fix so that the return for eeg are epochs 
    return epochs_array, fmri_array, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans


In [10]:

# 3. check last event for xtra tail 


In [11]:
def load_multiple_files(root: str,
                        rois: Optional[list[str]], eeg_channels: Optional[list[str]], sessions:list, starting_point_sec: int = 0,
                        rois_for_global_trend: Optional[list[str]] = None, roi_folder: str = 'roi'):
    datasets = {}
    event_ids_dict = {}
    if rois is None:
        overall_rois = None
    else:
        overall_rois = list(rois)

    if eeg_channels is None:
        overall_eeg_channels = None
    else:
        overall_eeg_channels = list(eeg_channels)

    for ses in sessions: 
        if ses == '1895':
            for task, run in RUNS_1895:
                #try:
                    epochs_array, fmri_array, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans = load_data(
                        root=root, ses=ses, run=run, task=task, rois=rois,
                        eeg_channels=eeg_channels, sampling_rates_ratio=500,
                        roi_folder=roi_folder
                    )

                    if overall_rois is None:
                        overall_rois = list(rois)
                    else:
                        assert overall_rois == rois

                    if overall_eeg_channels is None:
                        overall_eeg_channels = list(eeg_channels)
                    else:
                        assert overall_eeg_channels == eeg_channels

                    datasets[(eeg_path, fmri_path)] = {
                        'eeg': epochs_array, 'fmri': fmri_array, 'task': task.split('_')[0], 'eeg_times': eeg_times,'events':events, 'event_ids':event_ids,
                        'segment_scans':segment_scans
                    }
                    
                    event_ids_dict[(task, run, ses)] = event_ids
            #except Exception as e:
                #print(e) 
        else:
            for task, run in RUNS:
                #try:
                    epochs_array, fmri_array, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans = load_data(
                        root=root, ses=ses, run=run, task=task, rois=rois,
                        eeg_channels=eeg_channels, sampling_rates_ratio=500,
                        roi_folder=roi_folder
                    )

                    if overall_rois is None:
                        overall_rois = list(rois)
                    else:
                        assert overall_rois == rois

                    if overall_eeg_channels is None:
                        overall_eeg_channels = list(eeg_channels)
                    else:
                        assert overall_eeg_channels == eeg_channels

                    datasets[(eeg_path, fmri_path)] = {
                        'eeg': epochs_array, 'fmri': fmri_array, 'task': task.split('_')[0], 'eeg_times': eeg_times,'events':events, 'event_ids':event_ids,
                        'segment_scans':segment_scans
                    }
                    event_ids_dict[(task, run, ses)] = event_ids


    rois = list(rois)
    # if separate_global_trend:
    #     rois.append(' Global Regressive Trend')

    return datasets, rois, eeg_channels, event_ids_dict

In [34]:
root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1818', '1895']

datasets, rois, eeg_channels, event_ids_dict= load_multiple_files(root = root, sessions=['1818', '1895'],
                                                  rois = None, eeg_channels = None,  starting_point_sec=0,
                                                  rois_for_global_trend = None, roi_folder = 'roi')
     
    

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138060  =      0.000 ...   552.240 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']
Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138059  =      0.000 ...   552.236 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138055  =      0.000 ...   552.220 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']


...

Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138057  =      0.000 ...   552.228 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 96']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138024  =      0.000 ...   552.096 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 80', 'Stimulus/S 96']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped


In [35]:
datasets

{('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr',
  '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/roi/sub-101472_ses-1818_task-rs_run-01.csv'): {'eeg': array([[[ 5.62632751e-06,  3.86593652e-06,  5.60718012e-06, ...,
           -5.54979057e-05, -5.35919228e-05, -5.37890663e-05],
          [ 2.36492577e-05,  2.68408623e-05,  2.72661552e-05, ...,
            8.65679932e-05,  8.62720795e-05,  8.67350998e-05],
          [-4.68588524e-05, -4.56841469e-05, -4.63136940e-05, ...,
           -1.86672878e-05, -1.30011415e-05, -1.20530252e-05],
          ...,
          [ 3.68624344e-05,  2.68978500e-05,  2.50929508e-05, ...,
            1.23621239e-04,  1.31271545e-04,  1.36521500e-04],
          [ 1.60378586e-04,  1.41210938e-04,  1.17571999e-04, ...,
            1.07691154e-04,  1.03053474e-04,  1.15277969e-04],
          [ 4.63859634e-05,  4.58444443e-05,  4

In [13]:
for key, value in event_ids_dict.items(): 
    print(value['Scanner/Scan Start'] == 10002)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [14]:
for key, value in event_ids_dict.items():
    if  'socword' in key:
        print(key, value)

('socword', '01', '1818') {'New Segment/': 99999, 'Pulse Artifact/R': 10001, 'Scanner/Scan Start': 10002, 'Stimulus/S  1': 1, 'Stimulus/S  3': 3, 'Stimulus/S  5': 5, 'Stimulus/S  9': 9, 'Stimulus/S 10': 10, 'Stimulus/S 32': 32, 'Stimulus/S 48': 48, 'Stimulus/S 64': 64, 'Stimulus/S 80': 80, 'Stimulus/S 96': 96}
('socword', '02', '1818') {'New Segment/': 99999, 'Pulse Artifact/R': 10001, 'Scanner/Scan Start': 10002, 'Stimulus/S  1': 1, 'Stimulus/S  3': 3, 'Stimulus/S  5': 5, 'Stimulus/S  9': 9, 'Stimulus/S 10': 10, 'Stimulus/S 32': 32, 'Stimulus/S 48': 48, 'Stimulus/S 64': 64, 'Stimulus/S 80': 80, 'Stimulus/S 96': 96}
('socword', '03', '1818') {'New Segment/': 99999, 'Pulse Artifact/R': 10001, 'Scanner/Scan Start': 10002, 'Stimulus/S  1': 1, 'Stimulus/S  3': 3, 'Stimulus/S  5': 5, 'Stimulus/S  9': 9, 'Stimulus/S 10': 10, 'Stimulus/S 16': 16, 'Stimulus/S 32': 32, 'Stimulus/S 48': 48, 'Stimulus/S 64': 64, 'Stimulus/S 80': 80, 'Stimulus/S 96': 96}
('socword', '04', '1818') {'New Segment/': 

In [15]:

root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1818', '1895']


In [37]:
len(datasets.keys())

17

In [38]:
datasets[('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-socword_run-01_eegMRbvCBbviiR250.vhdr', '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/roi/sub-101472_ses-1818_task-socword_run-01.csv')]['eeg'].shape

(45, 127, 1250)

In [18]:
root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1804', '1818', '1895']
channels_dict_1804= {}
channels_dict_1818= {}
channels_dict_1895= {}

for task, run  in RUNS: 
    filename_1804= f'sub-101472_ses-{sessions[0]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'
    filename_1818= f'sub-101472_ses-{sessions[1]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'

    eeg_vhdr_path_1804 = os.path.join(root, f'ses-{sessions[0]}', 'eeg', filename_1804)
    eeg_vhdr_path_1818 = os.path.join(root, f'ses-{sessions[1]}', 'eeg', filename_1818)

    eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
    eeg_raw_1818 = mne.io.read_raw_brainvision(eeg_vhdr_path_1818, preload = True)

    df_eeg_1804 = eeg_raw_1804.to_data_frame()
    df_eeg_1818 = eeg_raw_1818.to_data_frame()

    channels_dict_1804[(task, run)] = df_eeg_1804.columns
    channels_dict_1818[(task, run)] = df_eeg_1818.columns


RUNS_1895 = [
    ('rs', '01'),
    ('rs', '02'),
    ('socword', '01'), # FAULTY FILE in eeg ses-1804 and ses-1895
    ('socword', '02'),
    ('socword', '03'),
    ]
for task, run  in RUNS_1895: 
    filename_1895= f'sub-101472_ses-{sessions[2]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'

    eeg_vhdr_path_1895 = os.path.join(root, f'ses-{sessions[2]}', 'eeg', filename_1895)

    eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)

    df_eeg_1895 = eeg_raw_1895.to_data_frame()

    channels_dict_1895[(task, run)] = df_eeg_1895.columns


    

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138063  =      0.000 ...   552.252 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138060  =      0.000 ...   552.240 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:14: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:14: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:15: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1818 = mn

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138062  =      0.000 ...   552.248 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138059  =      0.000 ...   552.236 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:14: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:14: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:15: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1818 = mn

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138068  =      0.000 ...   552.272 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138055  =      0.000 ...   552.220 secs...


...

/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138057  =      0.000 ...   552.228 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138024  =      0.000 ...   552.096 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_91439/2664388218.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


In [ ]:

root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
eeg_channels = None
sampling_rates_ratio = 500
leeway = 0.001
filename = f'sub-101472_ses-1818_task-socword_run-06_eegMRbvCBbviiR250.vhdr'
eeg_vhdr_path = os.path.join(root, f'ses-1818', 'eeg', filename)
eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)

events, event_ids = mne.events_from_annotations(eeg_raw)
events_dict_cut = {'Scanner/Scan Start': event_ids['Scanner/Scan Start']}  # use only Scan Start event



try:
    scan_key = 'Scanner/Scan Start'
    segment_scans = events[events[:, -1] == event_ids[scan_key]]
except: 
    scan_key = 'Stimulus/Scan Start'
    segment_scans = events[events[:, -1] == event_ids[scan_key]] 
    print(f'ERROR: FAULTY EVENT NAME. PATH:{eeg_vhdr_path}')


unique_scan_diffs, counts_diffs = np.unique(np.diff(segment_scans[:, 0]), return_counts=True)

first_segment = segment_scans[0, 0] / EEG_SAMPLING_RATE
cropped_eeg = eeg_raw.crop(tmin=first_segment)

df_eeg = eeg_raw.to_data_frame()
cropped_df_eeg = cropped_eeg.to_data_frame()

if eeg_channels is None:
    eeg_channels = [column for column in df_eeg.columns if column != 'time']

eeg_times = df_eeg['time']
cropped_eeg_times = cropped_df_eeg['time']

df_eeg = df_eeg[eeg_channels]


fmri_array, fmri_times, rois, fmri_path, df_fmri = load_fmri_data(
        root=root, run='01', ses='1818', task='rs', rois = None, roi_folder='roi'
    )





n_eeg_samples = df_fmri.shape[0] * sampling_rates_ratio
df_eeg_new = df_eeg.iloc[:n_eeg_samples]
eeg_array = df_eeg_new.to_numpy().T

epochs = mne.Epochs(
        cropped_eeg,
        events = segment_scans, 
        event_id =events_dict_cut, 
        tmin = 6.5,
        tmax = 11.5-1/EEG_SAMPLING_RATE, 
        baseline = None, 
        preload = True
    )
print(epochs.drop_log)

epochs_df = epochs.to_data_frame()
#epochs_df = epochs_df[:-1]
segment_scans = segment_scans[:-1]

#epochs = epochs.copy().filter(1, 70)
#epochs = epochs.crop(tmin=6 + leeway, tmax=12 - leeway, include_tmax=False)


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-socword_run-06_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138056  =      0.000 ...   552.224 secs...
Used Annotations descriptions: ['New Segment/', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 80', 'Stimulus/S 96']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/333210420.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/333210420.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ())


In [ ]:
epochs_df.to_numpy().shape

(57500, 130)

In [ ]:
# assert for tsart integer 
all([i.is_integer() for i in df_fmri[' Tstart']])
fmri_times = fmri_times.astype(int)
fmri_times

array([  0,   2,   4,  12,  14,  16,  24,  26,  28,  36,  38,  40,  48,
        50,  52,  60,  62,  64,  72,  74,  76,  84,  86,  88,  96,  98,
       100, 108, 110, 112, 120, 122, 124, 132, 134, 136, 144, 146, 148,
       156, 158, 160, 168, 170, 172, 180, 182, 184, 192, 194, 196, 204,
       206, 208, 216, 218, 220, 228, 230, 232, 240, 242, 244, 252, 254,
       256, 264, 266, 268, 276, 278, 280, 288, 290, 292, 300, 302, 304,
       312, 314, 316, 324, 326, 328, 336, 338, 340, 348, 350, 352, 360,
       362, 364, 372, 374, 376, 384, 386, 388, 396, 398, 400, 408, 410,
       412, 420, 422, 424, 432, 434, 436, 444, 446, 448, 456, 458, 460,
       468, 470, 472, 480, 482, 484, 492, 494, 496, 504, 506, 508, 516,
       518, 520, 528, 530, 532, 540, 542, 544])

In [ ]:
fmri_times = fmri_times//12
fmri_times.shape

(138,)

In [ ]:

df_fmri = df_fmri.assign(N_cycle=pd.Series(fmri_times).values)

In [ ]:
df_ncycle = df_fmri.groupby(by = 'N_cycle')
len(df_ncycle)

46

In [ ]:
rois

[' Left Cerebral White Matter',
 ' Left Cerebral Cortex',
 ' Left Lateral Ventricle',
 ' Left Thalamus',
 ' Left Caudate',
 ' Left Putamen',
 ' Left Pallidum',
 ' Left Brain-Stem',
 ' Right Brain-Stem',
 ' Left Hippocampus',
 ' Left Amygdala',
 ' Left Accumbens',
 ' Right Cerebral White Matter',
 ' Right Cerebral Cortex',
 ' Right Lateral Ventricle',
 ' Right Thalamus',
 ' Right Caudate',
 ' Right Putamen',
 ' Right Pallidum',
 ' Right Hippocampus',
 ' Right Amygdala',
 ' Right Accumbens']

In [ ]:
fmri_array = np.full((len(df_ncycle), len(rois), 3), fill_value = np.inf)

In [ ]:
for idx, group in df_ncycle: 
    group = group[rois].to_numpy()
    fmri_array[idx, ...] = group.T 

In [ ]:
# check if no inf that were previously filled are left  
assert not np.isinf(fmri_array).any()  ,(f'')  

<Epochs | 46 events (all good), 6 – 11.996 s (baseline off), ~67.0 MB, data loaded,
 'Scanner/Scan Start': 46>


In [21]:
def split_single_dataset(eeg: np.ndarray, fmri: np.ndarray, task: str): 
    
    # зафиксировать by % test and val, put the rest in train
    train_size = 25
    val_size = 10
    test_size = 10
    
    eeg_train, eeg_test, fmri_train, fmri_test = train_test_split(
        eeg, fmri, test_size=0.25, shuffle=False
        )

    eeg_train, eeg_val, fmri_train, fmri_val = train_test_split(
        eeg_train, fmri_train, test_size=0.2, shuffle=False
        ) 

    train_data = {'eeg': eeg_train,
                  'fmri': fmri_train}

    val_data = {'eeg': eeg_val,
                'fmri': fmri_val}

    test_data = {'eeg': eeg_test,
                 'fmri': fmri_test}

    return train_data, val_data, test_data

def split_datasets(datasets: dict):
    train_datasets = {}
    validation_datasets = {}
    test_datasets = {}

    for set_path, single_dataset in datasets.items():

        train_data, val_data, test_data = split_single_dataset(
            eeg=single_dataset['eeg'], fmri=single_dataset['fmri'],
            task=single_dataset['task']
        )

        train_datasets[set_path] = train_data

        if val_data is not None:
            validation_datasets[set_path] = val_data

        if test_data is not None:
            test_datasets[set_path] = test_data

    return train_datasets, validation_datasets, test_datasets




In [39]:
train_datasets, validation_datasets, test_datasets = split_datasets(datasets)

In [23]:
# initialise for each run and then concatDataset <- rewrite from build_full_dataset 
class EEGFMRIDataset(Dataset):
    def __init__(self, eeg, fmri, data_path, eeg_channels, fmri_rois):
        self.data_path = data_path
        self.eeg = torch.from_numpy(eeg).float()
        self.fmri = torch.from_numpy(fmri).float()

        self.eeg_channels = eeg_channels
        self.fmri_rois = list(fmri_rois)

        assert self.eeg.size(0) == self.fmri.size(0), (self.eeg.size(), self.fmri.size())

    def __getitem__(self, idx):
        return self.eeg[idx, ...], self.fmri[idx, ...]

    def __len__(self):
        return self.eeg.size(0)
    
    def get_rois(self):
        return list(self.fmri_rois)
    
def build_full_dataset(datasets:dict, fmri_rois:list[str]):
    torch_datasets = []
    all_rois = None
    for ds_path, dataset in datasets.items():
        torch_dataset = EEGFMRIDataset(
            eeg=dataset['eeg'], fmri=dataset['fmri'],
            eeg_channels=eeg_channels, fmri_rois=fmri_rois,
            data_path=ds_path)
        torch_rois = torch_dataset.get_rois()
        if all_rois is None:
            all_rois = list(torch_rois)
        else:
            assert all_rois == torch_rois, (all_rois, torch_rois)
        torch_datasets.append(torch_dataset)
    full_dataset = ConcatDataset(torch_datasets)

    return full_dataset, all_rois


In [60]:
datasets[('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr',
  '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/roi/sub-101472_ses-1818_task-rs_run-01.csv')]['eeg'].shape

(45, 127, 1250)

In [41]:
torch_datasets = {}
all_datasets = {'train': train_datasets, 'validation': validation_datasets, 'test': test_datasets}

for split_name in ['train', 'validation', 'test']:

    split_dataset, split_rois = build_full_dataset(
        datasets=all_datasets[split_name], fmri_rois=rois)

    torch_datasets[split_name] = {'dataset': split_dataset, 'rois': split_rois}

In [42]:
train_dataset, validation_dataset, test_dataset = torch_datasets['train']['dataset'], torch_datasets['validation']['dataset'], torch_datasets['test']['dataset']


In [76]:
test_dataset.datasets[2].eeg.shape


torch.Size([12, 127, 1250])

In [44]:
model_config = {"temporal_filter_kernel": 12, "dropout_prob": 0.25, "intermediate_channels": None}
scheduler_config = {"mode": "min", "factor": 0.1, "patience": 5}

TypeError: 'BrainWaveLinker' object is not subscriptable

In [45]:
brain_wave_linker_net = BrainWaveLinker(
        nn_parameters=model_config,
        eeg_channel_names=eeg_channels,
        roi_names=rois,
        criterion_name='MSELoss',
        criterion_kwargs={"dim": -1, "eps": 1e-06},
        optimizer_name='Adam',
        optimizer_kwargs={"lr": 0.001,"weight_decay": 0.001},
        scheduler_kwargs=scheduler_config
    )

In [ ]:
# 0.5 create git repository 
#  1. переписать bw_linker под окно размера 6s на входе  
# temporal_filter_kernel = 12 
# 2. for 3 datapoints kernel_size & stride == 4 
    # alternatively try temporal_filter_kernel = 10 
# 3. consider writing in torch instead of lightning 
    # in case of choosing lightning, rewrite metrics calculation
# 4. loss should be mse instead of neg_corr 

In [46]:
def build_dataloader(dataset: torch.utils.data.Dataset, sampler_parameters: Optional[dict],
                     dataloader_parameters: dict):

    samplers = {
        'RandomSampler': RandomSampler
    }

    if sampler_parameters is not None:
        sampler = samplers[sampler_parameters['name']](dataset, **sampler_parameters['parameters'])
    else:
        sampler = None

    dataloader = torch.utils.data.DataLoader(dataset, sampler=sampler, **dataloader_parameters)

    return dataloader


In [152]:

brain_wave_linker_net

BrainWaveLinkerSystem(
  (model): BrainWaveLinker(
    (spatial_filer): Conv1d(127, 22, kernel_size=(1,), stride=(1,))
    (pyramidal_subsampling): Sequential(
      (0): GELU(approximate='none')
      (1): Conv1d(22, 22, kernel_size=(4,), stride=(4,))
      (2): GELU(approximate='none')
      (3): Dropout(p=0.25, inplace=False)
      (4): Conv1d(22, 22, kernel_size=(4,), stride=(4,))
      (5): GELU(approximate='none')
      (6): Dropout(p=0.25, inplace=False)
      (7): Conv1d(22, 22, kernel_size=(4,), stride=(4,))
      (8): GELU(approximate='none')
      (9): Dropout(p=0.25, inplace=False)
    )
    (temporal_filter): Conv1d(22, 22, kernel_size=(12,), stride=(1,))
  )
  (criterion): MSELoss()
)

(26, 22, 3)

In [99]:
dataloader_configs = {"train": {"batch_size": 128,"drop_last": False,"num_workers": 0},  "validation": {
        "batch_size": 128,
        "drop_last": False,
        "num_workers": 0
    },
    "test": {
        "batch_size": 128,
        "drop_last": False,
        "num_workers": 0
    }
}

sampler_configs = {
        "train": None,
        "validation": None
    }

In [108]:
train_dataset.datasets[0].fmri.shape

torch.Size([26, 22, 3])

In [115]:
samplers = {'RandomSampler': RandomSampler}

train_loader = torch.utils.data.DataLoader(train_dataset, sampler=None, batch_size = 128, drop_last=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(validation_dataset, sampler=None, batch_size = 128, drop_last=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(train_dataset, sampler=None, batch_size = 128, drop_last=False, num_workers=0)





In [114]:
len(train_loader)

4

In [ ]:
# train_loader = build_dataloader(dataset=train_dataset, sampler_parameters=sampler_configs['train'],
#                                     dataloader_parameters=dataloader_configs['train'])
# val_loader = build_dataloader(dataset=validation_dataset, sampler_parameters=sampler_configs['validation'],
#                                   dataloader_parameters=dataloader_configs['validation'])
# test_loader = build_dataloader(dataset=test_dataset, sampler_parameters=None,
#                                    dataloader_parameters=dataloader_configs['test'])

In [116]:
print('train_loader', len(train_loader))
print('val_loader', len(val_loader))
print('test_loader', len(test_loader))

train_loader 4
val_loader 1
test_loader 4


In [ ]:
trainer_config= {
        "early_stopping": {
            "monitor": "val_loss",
            "mode": "min",
            "patience": 20
        },
        "checkpointing": {
            "monitor": "val_loss",
            "mode": "min",
            "save_top_k": 1
        },
        "kwargs": {
            "max_epochs": 250,
            "log_every_n_steps": 1
        }
    }

In [106]:
train_dataset.datasets[1].fmri.shape

torch.Size([26, 22, 3])

In [117]:
for i, (inputs, targets) in enumerate (train_loader): 
    print(f"Batch {i}:")
    print(f"Inputs shape: {inputs.shape}")
    print(f"Targets shape: {targets.shape}")

Batch 0:
Inputs shape: torch.Size([128, 127, 1250])
Targets shape: torch.Size([128, 22, 3])
Batch 1:
Inputs shape: torch.Size([128, 127, 1250])
Targets shape: torch.Size([128, 22, 3])
Batch 2:
Inputs shape: torch.Size([128, 127, 1250])
Targets shape: torch.Size([128, 22, 3])
Batch 3:
Inputs shape: torch.Size([58, 127, 1250])
Targets shape: torch.Size([58, 22, 3])


In [123]:
for batch in train_loader:
        inputs, targets = batch
        outputs = brain_wave_linker_net(inputs)  
        print(inputs.shape, outputs.shape, targets.shape, len(batch))

torch.Size([128, 127, 1250]) torch.Size([128, 22, 8]) torch.Size([128, 22, 3]) 2
torch.Size([128, 127, 1250]) torch.Size([128, 22, 8]) torch.Size([128, 22, 3]) 2
torch.Size([128, 127, 1250]) torch.Size([128, 22, 8]) torch.Size([128, 22, 3]) 2
torch.Size([58, 127, 1250]) torch.Size([58, 22, 8]) torch.Size([58, 22, 3]) 2


In [124]:
import lightning as L 

In [155]:
class BrainWaveLinker(nn.Module):
    
    def __init__(
            self, in_channels: int, out_channels: int, temporal_filter_kernel: int, dropout_prob: float = 0.25,
            intermediate_channels: Optional[int] = None
    ):
        super().__init__()

        if intermediate_channels is None:
            intermediate_channels = out_channels

        self.spatial_filer = nn.Conv1d(in_channels,  intermediate_channels, kernel_size=1)

        self.pyramidal_subsampling = nn.Sequential(
            nn.GELU(),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels,
                      kernel_size=4, stride=4),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels,
                      kernel_size=4, stride=4),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels,
                      kernel_size=4, stride=4),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        self.temporal_filter = nn.Conv1d(
            intermediate_channels, out_channels, kernel_size=temporal_filter_kernel, padding=0
        )

    def forward(self, x: torch.Tensor):
        out_sf = self.spatial_filer(x)
        out_ps = self.pyramidal_subsampling(out_sf)
        out = self.temporal_filter(out_ps)
        return out


class BrainWaveLinkerSystem(L.LightningModule):
    
    def __init__(self,
                 nn_parameters: dict,
                 roi_names: list[str], eeg_channel_names: list[str],
                 criterion_name: str, criterion_kwargs: dict,
                 optimizer_name: str, optimizer_kwargs: dict,
                 scheduler_kwargs: dict):
        super().__init__()

        self.model = BrainWaveLinker(
            in_channels=len(eeg_channel_names),
            out_channels=len(roi_names),
            **nn_parameters
        )
        self.criterion = build_criterion(criterion_name=criterion_name, criterion_kwargs=criterion_kwargs)
        self.eeg_channel_names = list(eeg_channel_names)
        self.roi_names = list(roi_names)
        self.criterion_name = criterion_name
        self.criterion_kwargs = criterion_kwargs
        self.optimizer_name = optimizer_name
        self.optimizer_kwargs = optimizer_kwargs
        self.scheduler_name = 'ReduceLROnPlateau'
        self.scheduler_kwargs = scheduler_kwargs
        self.full_metrics = {}
     

    def forward(self, inputs: torch.Tensor):
        return self.model(inputs)

    def log_metrics_tables(self, mode: str):
        assert self.full_metrics[mode] is not None, (self.full_metrics.keys(), mode)
        self.logger.log_table(key=f'{mode}_full_correlations',
                              dataframe=pd.DataFrame(self.full_metrics[mode]))

    def common_step(self, batch: tuple[torch.Tensor], batch_idx: int, mode: str):
        inputs, target = batch
        output = self(inputs)
        loss = self.criterion(output, target)
 
        self.log(f'{mode}_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def training_step(self, batch: tuple[torch.Tensor], batch_idx: int):
        loss = self.common_step(batch=batch,batch_idx=batch_idx,mode='train')
        return loss

    def validation_step(self, batch: tuple[torch.Tensor], batch_idx: int):
        loss = self.common_step(batch=batch,batch_idx=batch_idx,mode='val')
        return loss

    def test_step(self, batch: tuple[torch.Tensor], batch_idx: int):
        loss  = self.common_step(batch=batch,batch_idx=batch_idx,mode='test')
        return loss


    def configure_optimizers(self):
        optimizers = {
            'Adam': torch.optim.Adam,
            'AdamW': torch.optim.AdamW,
            'RMSprop': torch.optim.RMSprop,
            'SGD': torch.optim.SGD
        }
        optimizer = optimizers[self.optimizer_name](self.model.parameters(), **self.optimizer_kwargs)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, **self.scheduler_kwargs),
                # ReduceLROnPlateau scheduler requires a monitor
                'monitor': 'val_loss',
                'frequency': 1
            },
        }


In [128]:
def build_dataloader(dataset: torch.utils.data.Dataset, sampler_parameters: Optional[dict],
                     dataloader_parameters: dict):

    samplers = {
        'RandomSampler': RandomSampler
    }

    if sampler_parameters is not None:
        sampler = samplers[sampler_parameters['name']](dataset, **sampler_parameters['parameters'])
    else:
        sampler = None

    dataloader = torch.utils.data.DataLoader(dataset, sampler=sampler, **dataloader_parameters)

    return dataloader


In [ ]:
def extract_predicted_time_series(model: torch.nn.Module, ckpt_path: str, dataset: torch.utils.data.ConcatDataset,
                                  ds_name: str, device: str):
    # Load checkpoint
    checkpoint = torch.load(ckpt_path, weights_only=True, map_location=device)
    state_dict = {}

    # Original checkpoint for a torch.nn.Module was inside a lightning module, therefore removing additional prefix
    for key, value in checkpoint['state_dict'].items():
        key = key.split('.')
        assert key[0] == 'model'
        new_key = '.'.join(key[1:])
        state_dict[new_key] = value
    model.load_state_dict(state_dict)
    model.eval()

    # Check ROI consistency
    roi_names = None
    for ds in dataset.datasets:
        if roi_names is None:
            roi_names = list(ds.get_rois())
        else:
            assert roi_names == list(ds.get_rois()), (roi_names, ds.get_rois())


    for ds in dataset.datasets:

        eeg, gt_fmri, (eeg_data_path, fmri_data_path) = ds.get_all_data()
        eeg = eeg.unsqueeze(0)  # imitating batch dimension
        eeg.to(device)

        with torch.no_grad():
            pred_fmri = model(eeg)
            assert pred_fmri.size(0) == 1  # remove batch dimension
            pred_fmri = pred_fmri[0, ...]

        pred_fmri = pred_fmri.cpu().numpy()
        gt_fmri = gt_fmri.cpu().numpy()

        

        gt_fmri = gt_fmri[:, :-20]

        ds_results = {}

        for roi_idx, roi_name in enumerate(roi_names):
            roi_pred = pred_fmri[roi_idx, :]
            roi_gt = gt_fmri[roi_idx, :]
            ds_results[f'{roi_name}_pred'] = roi_pred
            ds_results[f'{roi_name}_gt'] = roi_gt


        ds_results = pd.DataFrame(ds_results)



In [138]:
def run_full_pipeline(trainer_config):


    callbacks = [ModelCheckpoint(**trainer_config['checkpointing'])]

    
    trainer = L.Trainer(callbacks=callbacks,
                        **trainer_config['kwargs'])

    best_model_state = None


    trainer.fit(model=brain_wave_linker_net, train_dataloaders=train_loader, val_dataloaders=val_loader)
    best_model_state = trainer.checkpoint_callback.best_model  # Assuming this retrieves the best model's state_dict


    print(f'Training complete')
    brain_wave_linker_net.load_state_dict(best_model_state)


    trainer.test(brain_wave_linker_net, dataloaders=test_loader)    

    # for logging_mode in config['dataframe_logging_modes']:
    #     brain_wave_linker_net.log_metrics_tables(mode=logging_mode)

    if torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'
        
    # extract_predicted_time_series(
    #     model=brain_wave_linker_net.model, ckpt_path=best_model_path, dataset=test_loader.dataset, ds_name='test',
    #     device=device
    # )


    # print_wrapped_text(header_text='MODEL', main_body=brain_wave_linker_net.model)
    # print_wrapped_text(header_text='Experiment config', main_body=config)
    # print_wrapped_text(header_text='Criterion config', main_body=criterion_config)
    # print_wrapped_text(header_text='Optimizer config', main_body=optimizer_config)
    # print_wrapped_text(header_text='Scheduler config', main_body=scheduler_config)
    # print_wrapped_text(header_text='Trainer config', main_body=trainer_config)
    # print_wrapped_text(header_text='wandb config', main_body=wandb_kwargs)
    # print_wrapped_text(header_text='Model config', main_body=model_config)


In [151]:
brain_wave_linker_net = BrainWaveLinkerSystem(
        nn_parameters=model_config,
        eeg_channel_names=eeg_channels,
        roi_names=rois,
        criterion_name='MSELoss',
        criterion_kwargs={},
        optimizer_name='Adam',
        optimizer_kwargs={"lr": 0.001,"weight_decay": 0.001},
        scheduler_kwargs=scheduler_config)

In [153]:
type(brain_wave_linker_net)

__main__.BrainWaveLinkerSystem

In [139]:
run_full_pipeline(trainer_config=trainer_config )

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `BrainWaveLinker`

/Users/terekhova/fsl/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([128, 22, 3])) that is different to the input size (torch.Size([128, 22, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (8) must match the size of tensor b (3) at non-singleton dimension 2

In [ ]:


def build_criterion(criterion_name: str, criterion_kwargs: dict):
    """
    Initializes a criterion (loss function) and returns it

    Parameters
    ----------
    criterion_name : str
        A name of the required criterion
    criterion_kwargs : dict
        A dict with keyword arguments for a desired criterion

    Returns
    -------
    criterion : torch.nn.Module
        An initialized criterion
    """
    if criterion_name == 'MSELoss':
        criterion = nn.MSELoss(**criterion_kwargs)
    elif criterion_name == 'L1Loss':
        criterion = nn.MSELoss(**criterion_kwargs)
    elif criterion_name == 'NegativeCorrelation':
        criterion = NegativeCorrelation(**criterion_kwargs)
    else:
        raise NotImplementedError
    return criterion